---
title: "2023_Kronensicherung_Plesse_001_Versuchsablauf"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Kraefte_Schwingungen
## Baumdaten, Gerädeanordung und Versuchsaufzeichung

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment) und installiere die Pakete linescale3 (LS3) und treeqinetic (PTQ) inklusive kj_core und kj_logger und weiteren requirements.

## Arbeitsumgebung vorbereiten

Es werden zuerst benötigte Standard-Pakete importiert. Nachfolgend die zwei extra geschriebenen Pakete LS3 und PTQ. Fehler beim Import dieser zwei Pakete sind ggf. Bugs. Beide Pakete nutzen eine gemeinsame CodeBasis in den Paketen kj_core (Core-Package) und kj_logger (individualisiertes Logging des Verarbeitungs-Prozesses). Diese sollte i. d. R. über die requirements mit installiert werden.

### IMPORT: Importieren von Standardbibliotheken

Die folgenden Bibliotheken werden importiert, um grundlegende Funktionen für Strukturierung, Datenverarbeitung, Plotting und statistische Auswertung bereit zu stellen.

In [1]:
# Struktur
from pathlib import Path
from typing import Dict, List

# Datenverarbeitung
import json
from IPython.display import Markdown, display
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from slugify import slugify  # Slugify ums strings in standard Formate zu überführen

Lade allgemeine Export-Funktionen, um die Daten als Latex-Tabellen zu exportieren

In [2]:
from kj_core.utils.latex_export import (
    save_latex_table,
    build_data_dict_df
)

### IMPORT: Importieren von eigenen Modulen

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [3]:
# Importiere alle Einstellungen aus der project_config.py
from project_config import (
    analyse_name,
    data_path,
    working_directory,
    data_export_directory,
    latex_export_directory
)



## Vorbemerkung

Die Daten wurden während des Versuchs als GoogleSheet erfasst und abschließend als CSV-Dateien exportiert, um ein einfaches Einlesen zur ermöglichen.
Nachfolgend werden Datentypen für alle Spalten explizit in den *_data_dict.json angegeben, da es teils durch Python-Pandas zu Fehlerkennungen kommt.
Zusätzlich werden teils Spaltennamen vereinheitlicht oder benannt.

## Daten des Versuchsbaums (tree_df)

### IMPORT: Daten und Datendokumentation

Die `tree.csv` liefert spezifische Informationen über den Versuchsbaum. Da nur ein Versuchsbaum, sind hier Daten und Datendokumentation in einer Datei.

In [4]:
tree_df = pd.read_csv(
    data_path / 'tree.csv',
    sep=';', decimal=',',
    na_values='NA',
    dtype='string'  # alle Spalten als string
)

### ANALYSE: Explorative Datenanalyse

In [5]:
#tree_df

In [6]:
# In Markdown umwandeln und anzeigen
md_text = tree_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|    | Variable            | Kategorie   | Zeichen                | Deutsch                        | Datentyp   | Einheit   | Beschreibung                        | Wert            |
|----|---------------------|-------------|------------------------|--------------------------------|------------|-----------|-------------------------------------|-----------------|
|  0 | baumart             | tree        | Baumart                | Baumart                        | string     | -         | Art des Baumes                      | Fagus silvatica |
|  1 | belaubung           | tree        | Belaubung              | Belaubung                      | string     | -         | Belaubungszustand zur Messung       | unbelaubt       |
|  2 | vitalitaet          | tree        | Vitalitaet             | Vitalität nach A. Roloff       | Int64      | Stufe     | Vitalitätsstufe nach Andreas Roloff | 1               |
|  3 | h_st_a              | tree        | $h_{\mathrm{StA}}$     | Höhe Stämmling A               | float64    | m         | Gesamthöhe des Stämmlings A         | 26.15           |
|  4 | h_st_b              | tree        | $h_{\mathrm{StB}}$     | Höhe Stämmling B               | float64    | m         | Gesamthöhe des Stämmlings B         | 27.20           |
|  5 | u_1m                | tree        | $u_{1\,\mathrm{m}}$    | Umfang auf 1~m                 | float64    | cm        | Stammumfang auf 1~m Höhe            | 140.00          |
|  6 | h_zwiesel           | tree        | $h_{\mathrm{Zwiesel}}$ | Höhe Zwiesel                   | float64    | m         | Höhe des Zwiesels über Boden        | 10.31           |
|  7 | h_ks                | tree        | $h_{\mathrm{KS}}$      | Höhe Kronensicherung           | float64    | m         | Höhe der KS-Anbringung über Boden   | 17.40           |
|  8 | l_ks                | tree        | $l_{\mathrm{KS}}$      | Länge Kronensicherung          | float64    | cm        | Länge der eingebauten KS            | 135.00          |
|  9 | u_st_a_h_ks         | tree        | $u_{\mathrm{StA,KS}}$  | Umfang Stämmling A auf Höhe KS | float64    | cm        | Stammumfang von A auf Höhe der KS   | 58.00           |
| 10 | u_st_b_h_ks         | tree        | $u_{\mathrm{StB,KS}}$  | Umfang Stämmling B auf Höhe KS | float64    | cm        | Stammumfang von B auf Höhe der KS   | 48.00           |
| 11 | standort_h_nn       | tree        | $h_{\mathrm{NN}}$      | Standort Höhe über n. N.       | float64    | m         | Standorthöhe über Normalnull        | 352.00          |
| 12 | standort_geo_breite | tree        | $\varphi$              | Standort Geo. Breite           | float64    | Grad      | Geografische Breite des Standorts   | 51.589476       |
| 13 | standort_geo_laenge | tree        | $\lambda$              | Standort Geo. Länge            | float64    | Grad      | Geografische Länge des Standorts    | 9.985242        |

### EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv)

In [7]:
# DataFrame als Feather
tree_df.to_feather(data_export_directory / "tree.feather")
tree_df.to_csv(data_export_directory / "tree.csv", sep=";", index=False, encoding="utf-8")

### LATEX-EXPORT: Daten und Datendokumentation als Latex-Tabelle exportieren (.tex)

In [8]:
select_data_fields = ["Zeichen", "Deutsch", "Wert", "Datentyp", "Einheit"]
tree_df_latex = tree_df[select_data_fields].copy()

latex_string = tree_df_latex.to_latex(index=False, escape=False, column_format="llrrr",
                                      float_format="{:0.2f}".format)
caption = "Feldversuch 2 - Daten des Versuchsbaums"

save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_daten_des_versuchsbaums.tex


## Daten der Geräteanordnung am Baum (sensor_df)

### IMPORT: Daten und Datendokumentation

Die Datendokumentation ergibt sich aus `sensor_position_data_dict.json`.

Die `sensor_position.csv` enthält detaillierte Informationen zur Anordnung der Sensoren an den Bäumen im Rahmen des Experiments. Jede Zeile beschreibt die Platzierung eines Sensors.

In [9]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path/ "sensor_position_data_dict.json", "r", encoding="utf-8") as f:
    sensor_data_dict = json.load(f)

In [10]:
# Erzeuge dtype_dict dynamisch aus dem der Daten Dokumentation
dtype_dict = {key: value["Datentyp"] for key, value in sensor_data_dict.items()
    if value["Datentyp"] not in [None, ""]
}

# CSV einlesen mit dynamischen Datentypen
sensor_file = data_path / 'sensor_position.csv'
sensor_df = pd.read_csv(sensor_file, sep=';', decimal=',', na_values='NA',dtype=dtype_dict)

### ANALYSE: Explorative Datenanalyse

In [11]:
# In DataFrame umwandeln
sensor_data_dict_df = build_data_dict_df(sensor_data_dict)

# In Markdown umwandeln und anzeigen
md_text = sensor_data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|    | Variable      | Kategorie       | Zeichen       | Deutsch     | Datentyp   | Einheit   | Beschreibung                             |
|----|---------------|-----------------|---------------|-------------|------------|-----------|------------------------------------------|
|  0 | location      | sensor_position | location      | Position    | string     | -         | Position des Sensors am Stamm            |
|  1 | height        | sensor_position | $h$           | Höhe        | Float64    | m         | Höhe des Sensors am Stamm                |
|  2 | diameter      | sensor_position | $d$           | Durchmesser | Float64    | cm        | Durchmesser des Stammes                  |
|  3 | direction     | sensor_position | direction     | Richtung    | string     | -         | Zug- oder Druckseite                     |
|  4 | type          | sensor_position | type          | Typ         | category   | -         | Typ des Sensors                          |
|  5 | sensor_id     | sensor_position | sensor\_id    | Sensor ID   | category   | -         | ID des Sensors                           |
|  6 | circumference | sensor_position | $u$           | Umfang      | Float64    | m         | Umfang des Stammes am Sensor             |
|  7 | position_id   | sensor_position | $position_\id | Position ID | Int64      | -         | Eindeutige ID für Sensorposition am Baum |
|  8 | note          | sensor_position | note          | Anmerkung   | string     | -         | Anmerkung zur Sensorposition             |

In [12]:
sensor_df["height"] = sensor_df["height"] / 100
sensor_df["circumference"] = sensor_df["circumference"] / 100
sensor_df["diameter"] = sensor_df["circumference"] / np.pi

sensor_df.head()

,position_id,type,sensor_id,location,height,circumference,direction,note,diameter
0,1,LS3,14:BF:E6,rope,18.3,<NA>,<NA>,<NA>,<NA>
1,2,LS3,14:99:1E,cable,17.4,<NA>,<NA>,<NA>,<NA>
2,3,TMS1,015,StA,18.0,0.45,west,<NA>,0.143239
3,4,TMS1,014,StB,18.0,0.4,west,<NA>,0.127324
4,5,TMS1,013,StA,15.0,0.67,west,<NA>,0.213268


### EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [13]:
# DataFrame als Feather
sensor_df.to_feather(data_export_directory / "sensor.feather")
sensor_df.to_csv(data_export_directory / "sensor.csv", sep=";", index=False, encoding="utf-8")

# Dict als JSON (UTF-8, sauber eingerückt)
with open(data_export_directory / "sensor_data_dict.json", "w", encoding="utf-8") as f:
    json.dump(sensor_data_dict, f,  indent=4, ensure_ascii=False)

### LATEX-EXPORT: Daten und Datendokumentation als Latex-Tabelle exportieren (.tex)

In [14]:
sensor_select_variables = ["type", "sensor_id", "location", "direction", "height", "circumference", "diameter"]

In [15]:
sensor_df_latex = sensor_df.copy()[sensor_select_variables]

# Ersetze Spaltennamen durch Formelzeichen
sensor_df_latex.columns = [sensor_data_dict[col]["Zeichen"] for col in sensor_select_variables]

latex_string = sensor_df_latex.to_latex(index=False, escape=False, column_format="lllrrrr",
    float_format="{:0.2f}".format)

caption = "Feldversuch 2 - Geräteanordnung"
save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_gerateanordnung.tex


In [16]:
sensor_data_dict_df = build_data_dict_df(sensor_data_dict, keys=sensor_select_variables, escape_index=True, select_latex_fields=True)
latex_string = sensor_data_dict_df.to_latex(index=False, escape=False)
caption = "Feldversuch 2 - Geräteanordnung Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_gerateanordnung_daten_dokumentation.tex


## Daten des Versuchsablaufs (series_df)

### IMPORT: Daten und Datendokumentation

Die Datendokumentation ergibt sich aus `series_data_dict.json`.

Die `series.csv` enthält das Protokoll des Versuchsablaufes der einzelnen Messungen. Jede Zeile repräsentiert eine einzelne Messung. 

In [17]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path/ "series_data_dict.json", "r", encoding="utf-8") as f:
    series_data_dict = json.load(f)

In [18]:
# Erzeuge dtype_dict dynamisch aus dem der Daten Dokumentation
dtype_dict = {key: value["Datentyp"] for key, value in series_data_dict.items()
    if value["Datentyp"] not in [None, ""]
}

# CSV einlesen mit dynamischen Datentypen
series_file = data_path / 'series.csv'
series_df = pd.read_csv(series_file, sep=';', decimal=',', na_values='NA', dtype=dtype_dict)

# Zeitspalte in Uhrzeit umwandeln
series_df["time"] = pd.to_datetime(series_df["time"], format="%H:%M:%S").dt.time

In [19]:
# Behandlungskategorien sortiert definieren
treatment_order = ['free', 'gefa_dynamic', 'cobra_static', 'cobra_static_slack']
treatment_category = CategoricalDtype(categories=treatment_order, ordered=True)
series_df["treatment"] = series_df["treatment"].astype(treatment_category)

### ANALYSE: Explorative Datenanalyse

Behandlungsvariante/Kronensicherung Kategorien:

- **`free`**: Der Baum konnte frei ohne Kronensicherung nach dem Release ausschwingen.
  
- **`gefa_dynamic`**: In ca. 2/3 der Baumhöhe wurde ein dynamisches Gefa Gurtband 4t dynamisch nach ZTV-Baumpflege mit leichtem Durchhang installiert. Das Ausschwingen wurde durch die KS abgedämpft. Da die Sicherung ohne Vorspannung installiert wurde, zeigen die Plots (/ls3/plots/force_vs_time_1/) von '14:99:1E' sowohl am Anfang als auch am Ende ca. 0 kN Kraft an (LogNr 1 bis 9).

- **`cobra_static`**: In ca. 2/3 der Baumhöhe wurde eine statische Cobra ultrastatic 7t (Dyneema) Sicherung installiert. Die Vorspannung betrug ca. 0,4 kN, wie sich in den Plots (/ls3/plots/force_vs_time_1/) von '14:99:1E' gut erkennen lässt. Durch das Zusammenziehen der Stämmlinge ist die Kronensicherung vor dem Release vollständig lastfrei (0 kN). Nach dem Release pendelt sich die Kraft ca. bei 0,4 kN ein (LogNr 10-18).

- **`cobra_static_slack`**: Ähnlich der `cobra_static`, jedoch wurde die Vorspannung entfernt. Aufgrund von Regen wurde nur eine Messung durchgeführt und die Serie frühzeitig abgebrochen.


In [20]:
# In DataFrame umwandeln
series_data_dict_df = build_data_dict_df(series_data_dict)

# In Markdown umwandeln und anzeigen
md_text = series_data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|    | Variable             | Kategorie   | Zeichen                               | Deutsch            | Datentyp   | Einheit   | Beschreibung                                                              |
|----|----------------------|-------------|---------------------------------------|--------------------|------------|-----------|---------------------------------------------------------------------------|
|  0 | treatment            | series      | treatment                             | Behandlung         | category   | -         | Art der KS: \texttt{free}, \texttt{gefa\_dynamic}, \texttt{cobra\_static} |
|  1 | release_force_target | series      | $F_{\mathrm{release,target}}$         | Vorspannkraft-Soll | float64    | kN        | Geplante Vorspannkraft im Zugseil bei Release                             |
|  2 | id                   | series      | id                                    | ID Messung         | Int64      | -         | ID der Messung / Beobachtung                                              |
|  3 | time                 | series      | $t$                                   | Zeit               | string     | Zeit      | ca. Uhrzeit Beginn                                                        |
|  4 | 14:BF:E6             | series      | $\mathrm{LogNr\ LS3}_{\mathrm{Seil}}$ | LogNr. LS3 Zugseil | Int64      | -         | LogNr. LineScale3 im Zugseil                                              |
|  5 | 14:99:1E             | series      | $\mathrm{LogNr\ LS3}_{\mathrm{KS}}$   | LogNr. LS3 KS      | Int64      | -         | LogNr. LineScale3 in KS                                                   |

In [21]:
series_df.head()

,id,time,release_force_target,treatment,14:BF:E6,14:99:1E
0,1,10:10:00,2.5,free,1,<NA>
1,2,10:20:00,2.8,free,2,<NA>
2,3,10:28:00,2.8,free,3,<NA>
3,4,10:47:00,2.8,free,4,<NA>
4,5,10:53:00,2.4,free,5,<NA>


### EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [22]:
# DataFrame als Feather
series_df.to_feather(data_export_directory / "series.feather")
series_df.to_csv(data_export_directory / "series.csv", sep=";", index=False, encoding="utf-8")

# Dict als JSON (UTF-8, sauber eingerückt)
with open(data_export_directory / "series_data_dict.json", "w", encoding="utf-8") as f:
    json.dump(series_data_dict, f,  indent=4, ensure_ascii=False)

### LATEX-EXPORT: Daten und Datendokumentation als Latex-Tabelle exportieren (.tex)

In [23]:
series_select_variables = ["id", "time", "release_force_target", "treatment", "14:BF:E6", "14:99:1E"]

In [24]:
series_df_latex = series_df.copy()[series_select_variables]

# Ersetze Spaltennamen durch Formelzeichen
series_df_latex.columns = [series_data_dict[col]["Zeichen"] for col in series_select_variables]

series_df_latex["treatment"] = series_df_latex["treatment"].apply(
    lambda x: "\\texttt{" + str(x).replace("_", "\\_") + "}"
)

latex_string = series_df_latex.to_latex(index=False, escape=False, column_format="llrlrr",
                                        float_format="{:0.2f}".format)

caption = "Feldversuch 2 - Versuchsablauf"
caption_long = "Feldversuch 2 - Versuchsablauf"

save_latex_table(latex_string, caption, latex_export_directory, caption_long=caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_versuchsablauf.tex


In [25]:
series_data_dict_df = build_data_dict_df(series_data_dict, keys=series_select_variables, escape_index=True, select_latex_fields=True)

latex_string = series_data_dict_df.to_latex(index=False, escape=False)
caption = "Feldversuch 2 - Versuchsablauf Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_versuchsablauf_daten_dokumentation.tex


# CUT

### Combined: Latex-Export von Daten für Anhang

In [26]:
def generate_grouped_latex_tables(df_latex: pd.DataFrame, caption: str, column_format: str, group_by: str, latex_export_directory: Path) -> None:
    """
    Generate grouped LaTeX tables for each unique value in a specified column of a DataFrame.

    Parameters:
        df_latex (pd.DataFrame): The DataFrame with already formatted columns.
        caption (str): The caption for the LaTeX tables.
        column_format (str): The format for the LaTeX tables.
        group_by (str): The column name to group by.
        latex_export_directory (Path): The directory to save the LaTeX tables.

    Returns:
        None
    """
    try:
        # DataFrame grouped by specified column
        grouped = df_latex.groupby(group_by, observed=True)

        # LaTeX tables to be combined in a single file
        combined_tables = []

        for group, group_df in grouped:
            # Drop the group_by column
            group_df = group_df.drop(columns=[group_by])

            # Format 'id' column as string if it exists
            if 'id' in group_df.columns:
                group_df['id'] = group_df['id'].astype(str)

            # Calculate statistics
            mean_row = group_df.mean(numeric_only=True).rename('Mean')
            median_row = group_df.median(numeric_only=True).rename('Median')
            sd_row = group_df.std(numeric_only=True).rename('SD')

            # Combine stats with original DataFrame
            stats_df = pd.concat([group_df, mean_row.to_frame().T, median_row.to_frame().T, sd_row.to_frame().T])

            # Generate LaTeX string from DataFrame
            df_latex_string = stats_df.to_latex(
                index=True,
                escape=False,
                column_format=column_format,
                float_format="{:0.2f}".format
            )

            # Create caption and label for the group
            caption_text = create_caption(caption, f"{caption} für {slugify(group, separator=' ')}")
            label_clean = create_label(caption=caption, additional_label=group)

            # Generate LaTeX table
            latex_table = generate_latex_table(df_latex_string, caption_text, label_clean)

            combined_tables.append(latex_table)

        # Combine all tables and save to a single file
        if combined_tables:
            final_output = "\n\n".join(combined_tables)
            file_name = create_label(caption) + ".tex"
            save_to_file(final_output, latex_export_directory / file_name)

    except Exception as e:
        print(f"Error generating grouped LaTeX tables: {e}")


In [27]:
# Liste der benötigten Variablen
variables = ["id", "treatment", "sensor_name", "location", "height", "diameter", "rope_release", "max_strain", "calc_max_strain", "strain_difference"]

# DataFrame kopieren und die gewünschten Spalten auswählen
df_latex = df.copy()[variables]

# Spaltennamen mit den Kurzbezeichnungen (Zeichen) aus dem data_dict umbenennen
df_latex = df_latex.rename(columns={var: data_dict[var]["Zeichen"] for var in variables})
# Funktionsaufruf mit Beispielparametern
generate_grouped_latex_tables(
    df_latex=df_latex,
    caption="Feldversuch 2 - Ergebnisse, Vergleich gemessene und rechnerische Faserdehnung",
    column_format="lll|lrrr|rrr",
    group_by="treatment",
    latex_export_directory=latex_export_directory
)


NameError: name 'df' is not defined

In [ ]:
variables = ['id', 'sensor_name', 'treatment', 'release_force_target', 'rope_release', 'cable_max', 'max_strain', 'max_compression',]

# DataFrame kopieren und die gewünschten Spalten auswählen
df_latex = df.copy()[variables]

# Spaltennamen mit den Kurzbezeichnungen (Zeichen) aus dem data_dict umbenennen
df_latex = df_latex.rename(columns={var: data_dict[var]["Zeichen"] for var in variables})
# Funktionsaufruf mit Beispielparametern
generate_grouped_latex_tables(
    df_latex=df_latex,
    caption="Feldversuch 2 - Ergebnisse, Kräfte und Dehnungen",
    column_format="lrl|rrr|rr",
        group_by="treatment",
    latex_export_directory=latex_export_directory
)


In [ ]:
variables_2 = ['id', 'sensor_name', 'treatment', 'm_amplitude', 'm_amplitude_2', 'initial_amplitude', 'damping_coeff', 'damping_ratio', 'frequency_damped', 'frequency_undamped', 'y_shift', 'pearson_r', 'nmae']

# DataFrame kopieren und die gewünschten Spalten auswählen
df_latex = df.copy()[variables_2]

# Spaltennamen mit den Kurzbezeichnungen (Zeichen) aus dem data_dict umbenennen
df_latex = df_latex.rename(columns={var: data_dict[var]["Zeichen"] for var in variables_2})
# Funktionsaufruf mit Beispielparametern
generate_grouped_latex_tables(
    df_latex=df_latex,
    caption="Feldversuch 2 - Ergebnisse, Schwingungsparameter",
    column_format="lrl|rrr|rr|rr|r|rr",
    group_by="treatment",
    latex_export_directory=latex_export_directory
)

### PTQ: Analyse der Schwingungsparameter

In diesem Abschnitt werden die Schwingungsparameter statistisch ausgewertet. Ziel ist es, den Einfluss verschiedener Behandlungsvarianten (treatment) auf die gemessenen Schwingungsparameter zu untersuchen und dabei auch den potenziellen Einfluss der Vorspannung (rope_release) und Sensorposition (sensor_name) zu berücksichtigen.


In [ ]:
variables = [
    'm_amplitude', 
    'm_amplitude_2',
    'max_strain',
    'max_compression',
    'initial_amplitude',
    'damping_coeff', 
    'damping_ratio', 
    'frequency_damped', 
    'frequency_undamped',
    'pearson_r',
    #'nrmse', 
    'nmae'
]

#### Systematischer Einfluss der Sensorposition

Ziel: Visuell erkennen, ob unterschiedliche Sensoren konsistent andere Werte liefern und ob dieser Effekt die Interpretation der treatment-Effekte erschwert.

In [ ]:
# Plot erstellen
for var in variables:
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="sensor_name", y=var, data=df, hue="treatment", palette=treatment_color_dict,  dodge=True)
    # Stripplot: Punkte zur Veranschaulichung der Verteilung
    sns.stripplot(x="sensor_name", y=var, data=df, hue="treatment", palette=treatment_color_dict, dodge=True, alpha=1, jitter=True, size=5, legend=False)
    # Titel und Achsentitel setzen
    plt.title(f"Einfluss von treatment auf {var} gruppiert über sensor_name")
    plt.xlabel("Sensor Name")
    plt.ylabel(var)
    plt.tight_layout()
    #plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"effect_sensor_treatment_{var}", subdir="combined/sensor")

#### Systematischer Einfluss der Behandlungsvariante

Ziel: Feststellen, ob die Variation durch unterschiedliche Behandlungen relativ zur sensorbedingten Variation unterscheidbar ist.

In [ ]:
# Plot erstellen
for var in variables:
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="treatment", y=var, data=df, palette=sensor_color_dict, hue='sensor_name')
    sns.stripplot(x="treatment", y=var, data=df, palette=sensor_color_dict, hue='sensor_name', dodge=True, alpha=1, jitter=True, size=5, legend=False)
    plt.title(f"Einfluss von sensor_name auf {var} gruppiert über treatment")
    plt.xlabel("treatment")
    plt.ylabel(var)
    plt.tight_layout()
    #plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"effect_treatment_sensor_{var}", subdir="combined/treatment")

#### Versuche Mixed-Linear Model
Die Daten sind hierarchisch: Mehrere Messungen (vier Sensoren) pro Beobachtungseinheit (`id`). Ein Mixed-Effects Modell könnte diese Struktur abbilden, indem zufällige Effekte für `id` und feste Effekte für `treatment` sowie `sensor_name` genutzt werden. Zusätzlich könnte `rope_release` als Kovariate eingeführt werden.

Diese Modelle wären theoretisch präziser, aber aufgrund der geringen Stichprobengröße und der komplexen Datenstruktur treten Konvergenzprobleme auf.


In [ ]:
# Ergebnisse als Dictionary speichern
results_dict = {}

for var in variables:
    # Formel für MixedLM: Parameter ~ C(treatment) + C(sensor_name) + rope_release + (1|id)
    formula = f"{var} ~ C(treatment) + C(sensor_name)" #  +  rope_release
    model = smf.mixedlm(formula, data=df, groups=df["id"])
    
    # Modell fitten
    fit = model.fit(reml=True)  # REML ist Standard für gemischte Modelle
    
    # Ergebnisse ausgeben
    print(f"\n### Ergebnisse für {var} ###")
    print(fit.summary())
    
    # Überprüfen, ob das Modell konvergiert ist
    if not fit.converged:
        print("Achtung: Das Modell ist nicht konvergiert. Erwägen Sie Anpassungen (z.B. Skalierung der Daten).")

#### Vereinfachtes Vorgehen durch Aggregation (Mittelung über Sensoren)

Um dennoch aussagekräftige Aussagen zu erhalten, werden die Messungen pro `id` über alle Sensoren gemittelt. Dadurch geht zwar die Variation aufgrund unterschiedlicher Sensoren verloren, aber es entsteht ein stabileres Datenset, in dem jede `id` einen aggregierten Wert pro Parameter hat.

Auf dieser Basis können einfache OLS-Modelle geschätzt werden, z. B. `Parameter ~ C(treatment) + rope_release`. Diese Modelle sind einfacher und sollten stabil konvergieren.

In [ ]:
# Placeholder for results
results = {}

# Step 1: Compute the mean of each variable for each id
grouped_df = df.groupby("id").agg(
    {var: "mean" for var in variables}
).reset_index()

# Add additional variables (e.g., treatment and rope_release) to the aggregated data
grouped_df = grouped_df.merge(
    df[["id", "treatment", "rope_release"]].drop_duplicates(), on="id"
)

# Step 2: Fit linear models to the aggregated data
for var in variables:
    try:
        # Fit a linear model
        model = smf.ols(f"{var} ~ C(treatment) + rope_release", grouped_df).fit()

        # Store relevant results
        results[var] = {
            "p_value_treatment": model.pvalues.get("treatment[T.statisch]", None),
            "p_value_rope_release": model.pvalues.get("rope_release", None),
            "summary": model.summary()
        }
    except Exception as e:
        results[var] = {"error": str(e)}

# Display results
results

Nicht jeder Parameter wird durch `rope_release` beeinflusst. Nur für jene Parameter, bei denen ein signifikanter Einfluss von `rope_release` festgestellt wird, soll dieser Effekt herausgerechnet werden. Auf diese Weise entstehen "bereinigte" Werte, in denen der lineare Einfluss von `rope_release` entfernt ist.

In [ ]:
# Placeholder for results
# Dieses Dictionary wird nun sowohl die Modellobjekte als auch die Kennwerte speichern.
results = {}

# Liste der Variablen, für die rope_release berücksichtigt wird
relevant_vars_rope_release = ['m_amplitude', 'm_amplitude_2', 'max_strain', 'max_compression']

# Step 1: Compute the mean of each variable for each id
grouped_df = df.groupby("id").agg(
    {var: "mean" for var in variables}
).reset_index()

# Add additional variables (e.g., treatment and rope_release) to the aggregated data
grouped_df = grouped_df.merge(
    df[["id", "treatment", "rope_release"]].drop_duplicates(), on="id"
)

# Step 2: Fit linear models to the aggregated data and store model objects and p-Werte in results
for var in variables:
    # Formuliere das Modell dynamisch, abhängig davon, ob rope_release relevant ist
    if var in relevant_vars_rope_release:
        formula = f"{var} ~ C(treatment) + rope_release"
    else:
        formula = f"{var} ~ C(treatment)"

    try:
        # Fit a linear model
        model = smf.ols(formula, grouped_df).fit()

        # Store model and relevant results directly in results
        results[var] = {
            "model": model,
            "p_value_treatment": model.pvalues.get("treatment[T.statisch]", None),
            "p_value_rope_release": model.pvalues.get("rope_release", None) if var in relevant_vars_rope_release else None,
            "summary": model.summary()
        }
    except Exception as e:
        results[var] = {"error": str(e)}

# Display results
print(results)

In [ ]:
# Beispiel für m_amplitude
plt.figure(figsize=(10, 6))
sns.lmplot(data=grouped_df, x='rope_release', y='m_amplitude_2', hue='treatment', ci=95)
plt.title('Einfluss von rope_release auf m_amplitude für verschiedene Treatments')
plt.xlabel('rope_release (kN)')
plt.ylabel('m_amplitude')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=grouped_df, x='treatment', y='m_amplitude')
plt.title('Vergleich von m_amplitude zwischen den Treatments')
plt.xlabel('Treatment')
plt.ylabel('m_amplitude')
plt.show()


In [ ]:
import pandas as pd
from tabulate import tabulate
import re

# Liste der Variablen, die in die Tabelle übernommen werden sollen
variables_spec = [
    'm_amplitude_2',
    'max_compression',
    'damping_coeff',
    'damping_ratio',
    'frequency_damped',
    'frequency_undamped',
]

# Umbenennung der Spalten für LaTeX-Notation
column_rename_map = {
    'm_amplitude_2': r'$mA_2$',
    'max_compression': r'$\text{max\_C}$',
    'damping_coeff': r'$\delta$',
    'damping_ratio': r'$D$',
    'frequency_damped': r'$f_d$',
    'frequency_undamped': r'$f_0$',
}

# Funktion zur Erstellung der Modellgüte-Kennzahlen für alle Variablen
def create_model_metrics_table(variables, results):
    metrics_data = {
        "Kennzahl": ["R²", "Adj. R²", "F-St.", "AIC", "N"]
    }

    for var in variables:
        col_name = column_rename_map.get(var, var)  # Verwende gekürzten Namen, falls vorhanden
        if var not in results or 'error' in results[var]:
            metrics_data[col_name] = ["n/a"] * 5
        else:
            model = results[var]['model']
            metrics_data[col_name] = [
                f"{model.rsquared:.4f}",
                f"{model.rsquared_adj:.4f}",
                f"{model.fvalue:.4f}",
                f"{model.aic:.4f}",
                f"{model.nobs:.0f}"
            ]

    # Erstelle die Tabelle mit tabulate
    metrics_df = pd.DataFrame(metrics_data)
    return tabulate(
        metrics_df,
        headers="keys",
        tablefmt="latex_raw",
        floatfmt=".4f",
        showindex=False)

# Funktion zur Erstellung einer LaTeX-Tabelle für die Koeffizienten einer Variable
def create_latex_table_for_variable(var, results):
    if var not in results:
        return f"%% Keine Ergebnisse für Variable {var} vorhanden."
    
    model_result = results[var]
    if 'error' in model_result:
        return f"%% Fehler beim Anpassen des Modells für {var}: {model_result['error']}"
    
    model = model_result['model']
    summary = model.summary2().tables[1]  # Zugriff auf die Tabelle der Koeffizienten

    # Erstelle eine LaTeX-Tabelle mit tabulate für die Koeffizienten
    latex_table = tabulate(
        summary,
        headers=summary.columns,
        tablefmt="latex_booktabs",
        floatfmt=".4f",
        showindex=True
    )

    # Escape Unterstriche in der Variable für LaTeX
    escaped_var = re.sub(r'_', r'\_', var)
    shortened_var = column_rename_map.get(var, escaped_var)

    # Füge die LaTeX-Caption zur Tabelle hinzu
    return f"""
\\begin{{table}}[ht]
    \\centering
    \\caption{{Modellzusammenfassung für {escaped_var} ({shortened_var})}}
    \\begin{{adjustbox}}{{max width=\\textwidth}}
    {latex_table}
    \\end{{adjustbox}}
\\end{{table}}
"""

# Erstelle die Modellgüte-Tabelle für alle Variablen
latex_metrics_table = create_model_metrics_table(variables_spec, results)
print("""
\\begin{table}[ht]
    \\centering
    \\caption{Modellgüte für alle Variablen}
    \\begin{adjustbox}{max width=\\textwidth}
""")
print(latex_metrics_table)
print("""
    \\end{adjustbox}
\\end{table}

\\vspace{1cm}
""")

# Erstelle und print die LaTeX-Tabellen für die Koeffizienten jeder Variable
for var in variables_spec:
    latex_output = create_latex_table_for_variable(var, results)
    print(latex_output)

In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables), 2, (len(variables) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_with_rope_release", subdir="combined")


Für die betroffenen Parameter wird der Einfluss von `rope_release` mithilfe der bereits angepassten Modelle (`Parameter ~ C(treatment) + rope_release`) entfernt. Dazu werden Vorhersagen für einen konstanten `rope_release`-Wert (den Mittelwert) berechnet und mit den tatsächlichen Werten verglichen. Die daraus resultierenden bereinigten Werte sind frei von Variation, die auf `rope_release` zurückzuführen wäre.

In [ ]:
grouped_adjusted_df = grouped_df.copy()

# Bereinigung: Wir setzen rope_release auf seinen Mittelwert
rope_mean = grouped_df["rope_release"].mean()

for var in relevant_vars_rope_release:
    # Zugehöriges Modellobjekt abrufen
    model = results[var]["model"]

    # Vorhersage mit tatsächlichen rope_release-Werten
    predicted_current = model.predict(grouped_df)

    # Vorhersage, wenn rope_release = rope_mean gesetzt wird
    df_mean_rope = grouped_df.copy()
    df_mean_rope["rope_release"] = rope_mean
    predicted_mean = model.predict(df_mean_rope)

    # Angepasste Werte berechnen:
    actual = grouped_df[var].values
    adjusted = actual + (predicted_mean - predicted_current)
    grouped_adjusted_df[var] = adjusted

In [ ]:
grouped_df.describe()

In [ ]:
grouped_adjusted_df.describe()

##### Latex Tabelle Output


In [ ]:
data_dict

In [ ]:
# DataFrame mit den relevanten Spalten erstellen
variables_latex = [
    'm_amplitude', 
    'm_amplitude_2',
    'initial_amplitude',
    'damping_coeff', 
    'damping_ratio', 
    'frequency_damped', 
    'frequency_undamped',
    'pearson_r',
    'nmae'
]
df_latex = grouped_adjusted_df[variables_latex + ['treatment']].copy()
df_latex.columns

In [ ]:


# Spaltennamen entsprechend der LaTeX-Notation umbenennen
df_latex.rename(columns=data_dict, inplace=True)

# Erstellung deskriptiver Statistiken für alle Beobachtungen
overall_stats = df_latex.describe().drop(index=['count', '25%', '75%'])
overall_stats.rename(index={'50%': 'median'}, inplace=True)

# Erstellung der deskriptiven Statistiken für jede Gruppe
grouped_stats = {
    'overall': overall_stats
}

for treatment, group in df_latex.groupby('treatment', observed=True):
    group_stats = group.describe().drop(index=['count', '25%', '75%'])
    group_stats.rename(index={'50%': 'median'}, inplace=True)
    grouped_stats[treatment] = group_stats

# Zusammenführen der Statistiken in einer Tabelle
combined_stats = pd.concat(grouped_stats, names=['Treatment'])

# LaTeX-Export des kombinierten DataFrames
df_latex_string = combined_stats.to_latex(
    escape=False,
    multirow=True,
    multicolumn=True,
    column_format="l|lrrrrrrrrr", 
    float_format="{:0.2f}".format
)

# LaTeX-Tabellencode erstellen
latex_table = f"""
\\begin{{table}}[h]
    \\centering
    \\caption{{Feldversuch 2 - Ergebnisse, Schwingungsparameter deskriptive Statistiken (Gesamt und gruppiert über Treatment), Amplituden korrigiert über \\texttt{{rope\\_release}}, 9 Beobachtung je Gruppe, jeweils Mittelwert für 4 Elastometer}}
    \\begin{{adjustbox}}{{max width=\\textwidth}}
    {df_latex_string}
    \\end{{adjustbox}}
    \\label{{tab:Feldversuch_2_Deskriptive_Statistiken_Schwingungsparameter}}
\\end{{table}}
"""

print(latex_table)

#### Visualisierung der bereinigten Ergebnisse

Abschließend werden die bereinigten Werte grafisch dargestellt, um die Unterschiede zwischen den Behandlungen in Abwesenheit des `rope_release`-Einflusses zu verdeutlichen. Dies zeigt, wie sich die Treatments auf die Parameter auswirken würden, wenn für alle Einheiten die gleiche mittlere Vorspannung gelten würde.

In [ ]:
relevant_vars_rope_release = ['m_amplitude', 'm_amplitude_2', 'max_strain', 'max_compression']

n_vars = len(relevant_vars_rope_release)
n_cols = 2  # Links Original, rechts angepasst
n_rows = n_vars  # Eine Zeile pro Variable

fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))

for i, var in enumerate(relevant_vars_rope_release):
    # Linke Spalte: Original (grouped_df)
    sns.boxplot(ax=axes[i,0], x="treatment", y=var, data=grouped_df, 
                palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i,0], x="treatment", y=var, data=grouped_df, 
                  dodge=False, c="black", jitter=True, size=5)
    axes[i,0].set_title(f"original: {var}")
    axes[i,0].set_ylabel(var)

    # Rechte Spalte: Angepasst (grouped_adjusted_df)
    sns.boxplot(ax=axes[i,1], x="treatment", y=var, data=grouped_adjusted_df, 
                palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i,1], x="treatment", y=var, data=grouped_adjusted_df, 
                  dodge=False, c="black", jitter=True, size=5)
    axes[i,1].set_title(f"adjusted : {var}")
    axes[i,1].set_ylabel(var)
    
    # Y-Limits von links holen
    y_min, y_max = axes[i,0].get_ylim()
    # Y-Limits auf rechts anwenden
    axes[i,1].set_ylim(y_min, y_max)

plt.tight_layout()
plt.show()

PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_comparison", subdir="combined")


Schritt 2: Durchführung von Post-hoc-Tests

Festzustellen welche paarweisen Unterschiede zwischen den Treatments signifikant sind.

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Wir gehen davon aus, dass grouped_adjusted_df existiert und die Spalten 'treatment' sowie die Variablen aus 'variables' enthält.

for var in variables_spec:
    # Tukey HSD Test durchführen
    # Annahme: Die Spalte 'treatment' enthält die Gruppennamen z.B. 'free', 'gefa_dynamic', 'cobra_static'
    # pairwise_tukeyhsd benötigt die abhängige Variable und die Gruppen.
    tukey_results = pairwise_tukeyhsd(endog=grouped_adjusted_df[var],
                                      groups=grouped_adjusted_df['treatment'],
                                      alpha=0.05)
    
    print(f"--- Post-Hoc Test (Tukey HSD) für Variable: {var} ---")
    print(tukey_results.summary())
    print("\n")


In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables_spec), 2, (len(variables_spec) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables_spec):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_without_rope_release", subdir="combined")

In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables_spec), 2, (len(variables_spec) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 5 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables_spec):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_without_rope_release_spec", subdir="combined")

#### Vorhergesagte Werte extrahieren und Boxplots für die Sensoren erstellen

In [ ]:
# Vorhergesagte Werte aus den Modellen extrahieren
for variable in variables:
    df[f'predicted_{variable}'] = models[variable].fittedvalues

# Boxplots erstellen mit den vorhergesagten Werten
fig, axes = plt.subplots(4, 2, figsize=(8, 8))
axes = axes.flatten()

for i, variable in enumerate(variables):
    sns.boxplot(ax=axes[i], x='treatment', y=f'predicted_{variable}', data=df, palette=treatment_color_dict, hue='treatment', dodge=False, legend=False)
    axes[i].set_title(f'{variable} by Treatment')
    axes[i].set_xlabel('Treatment')
    axes[i].set_ylabel(f'Predicted {variable}')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="predicted_effect_for_treatment", subdir="combined")

In [ ]:
def annotate_tukey(ax, tukey_result, significance_level=0.05):
    """
    Fügt eine Textbox mit den Tukey-Test-Ergebnissen und dem festgelegten Signifikanzniveau in den Plot ein.
    
    Parameters:
    ax (matplotlib.axes): Die Achse, auf der der Plot gezeichnet wird.
    tukey_result (TukeyHSDResults): Die Ergebnisse des Tukey HSD Tests.
    significance_level (float): Das Signifikanzniveau, standardmäßig 0.05.
    """
    # Definiere die gewünschte Reihenfolge der Vergleiche
    comparisons_order = [('free', 'gefa_dynamic'), ('free', 'cobra_static'), ('gefa_dynamic', 'cobra_static')]

    # Text für die Annotation zusammenstellen
    text_str = f"Tukey HSD Results: \n(Significance level = {significance_level:.2f})\n\n"
    
    # Durchlaufe die gewünschte Vergleichsreihenfolge
    for group1, group2 in comparisons_order:
        # Filtere die korrekte Paarung aus den Tukey-Ergebnissen
        for i in range(len(tukey_result._results_table.data[1:])):
            pair = tukey_result._results_table.data[i + 1]
            if (pair[0] == group1 and pair[1] == group2) or (pair[0] == group2 and pair[1] == group1):
                p_value = tukey_result.pvalues[i]
                significance = "*" if p_value < significance_level else "n.s."
                text_str += f"\n{group1} vs {group2}: \np = {p_value:.4f} ({significance})\n\n"
    
    # Textbox am Rand des Plots hinzufügen
    ax.annotate(text_str, xy=(1.01, 0.1), xycoords='axes fraction', va='center', ha='left')

# Einzelne Plots für jede Variable erstellen und speichern
for variable in variables:
    fig, ax = plt.subplots(figsize=(8, 4))
    
    # Boxplot für die aktuelle Variable
    sns.boxplot(ax=ax, x='treatment', y=f'predicted_{variable}', data=df, 
                palette=treatment_color_dict, hue='treatment', dodge=False, legend=False)
    
    # Tukey-Test für die aktuelle Variable
    tukey_result = tukey_results[variable]
    
    # Tukey-Ergebnisse annotieren
    annotate_tukey(ax, tukey_result)
    
    ax.set_title(f'{variable} by Treatment')
    ax.set_xlabel('Treatment')
    ax.set_ylabel(f'Predicted {variable}')
    
    plt.tight_layout()
    plt.show()

    # Plot speichern
    plot_filename = f"{variable}_effect_for_treatment"
    PLOT_MANAGER.save_plot(fig, filename=plot_filename, subdir="osc_variables_box")


In [ ]:
# Gruppieren des DataFrames nach 'treatment' und Entfernen unnötiger Spalten
df_treatment_describe = (df.drop(['id', 'ptq_sensor_name'], axis=1)
                         .groupby('treatment', observed=True)
                         .describe())

df_treatment_describe = df_treatment_describe.reset_index()
df_treatment_describe.round(2)

In [ ]:
df.dtypes

In [ ]:
# Gruppieren des DataFrames nach 'treatment' und Entfernen unnötiger Spalten
df_sensor = (df.drop(['id', 'release_force_target', 'ls3_rope_release', 'ls3_cable_max', 'location', 'height', 'diameter', 'direction'], axis=1).
             groupby(['treatment', 'ptq_sensor_name'], observed=True).
             mean())  #.T
#df_sensor.round(4)

In [ ]:
# Gruppieren des DataFrames und Anwenden von mean für ptq_sensor_name 
df_id = ((df.drop(['ptq_sensor_name', 'location', 'height', 'diameter', 'direction'], axis=1)
          .groupby(['treatment', 'id'], observed=True)
          .mean())
         .reset_index())

df_id.round(4).head()

### Zusammenhangsanalyse für LS3 und PTQ

In [ ]:
# Auswahl der neuen Spaltennamen für die Korrelationsmatrix
columns_corr = ['ptq_m_amplitude',
                'ptq_m_amplitude_2',
                'ptq_initial_amplitude',
                'ptq_damping_coeff',
                'ptq_angular_frequency',
                'ptq_y_shift',
                'ptq_pearson_r',
                #'ptq_nrmse',
                'ptq_nmae',
                'release_force_target',
                'ls3_rope_release',
                'ls3_cable_max']
df_corr = df_id.copy()[columns_corr]

# Berechnung der Korrelationsmatrix
correlation_matrix = df_corr.corr()

# Visualisierung der Korrelationsmatrix mit Seaborn
fig1, ax = plt.subplots(figsize=(8, 8))  # Anpassen der Größe der Grafik
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', ax=ax, annot_kws={'size': 10})

# Titel und Schriftgrößen anpassen
#plt.title('Correlation Matrix for LS3 and PTQ', fontsize=18)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize=12, rotation=45, ha='right')
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize=12, rotation=45, ha='right')
ax.set_xlabel('')
ax.set_ylabel('')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig1, filename="correlation_matrix", subdir="combined")

In [ ]:
# ANOVA für 'ls3_rope_release'
model_rope_release = smf.ols('ls3_rope_release ~ treatment', data=df_id).fit()
anova_rope_release = sm.stats.anova_lm(model_rope_release, typ=2)

# ANOVA für 'ls3_cable_max'
model_cable_max = smf.ols('ls3_cable_max ~ treatment', data=df_id).fit()
anova_cable_max = sm.stats.anova_lm(model_cable_max, typ=2)

# Zusammenfassungen der Modelle
summary_rope_release = model_rope_release.summary()
summary_cable_max = model_cable_max.summary()
# Zusammenfassungen der Modelle
summary_rope_release_latex = model_rope_release.summary().as_latex()
summary_cable_max_latex = model_cable_max.summary().as_latex()

#print(summary_rope_release_latex)
#print(summary_cable_max_latex)

anova_rope_release, summary_rope_release, anova_cable_max, summary_cable_max

In [ ]:
# Funktion zur Erstellung des Text-Strings für die statistischen Parameter
def annotate_stats(x, y):
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    text_str = f"R = {r_value:.2f}\nSlope = {slope:.2f}\nIntercept = {intercept:.2f}\np-value = {p_value:.2e}\nStd Err = {std_err:.2f}"
    return text_str

#### Zusammenhang Ziel- und Ist-Vorspannung

In [ ]:
fig2, ax1 = plt.subplots(figsize=(8, 5))
sns.regplot(x='release_force_target', y='ls3_rope_release', data=df_id, ax=ax1, color='b', ci=95)
ax2 = ax1.twinx()
ax2.axis('off')
stats_text = annotate_stats(df_id['release_force_target'], df_id['ls3_rope_release'])
ax2.annotate(stats_text, xy=(1.01, 0.1), xycoords='axes fraction')
#ax1.set_title('Correlation Between Target and Actual Release Forces"')
ax1.set_xlabel('Release Force Target [kN]')
ax1.set_ylabel('Release Force [kN]')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig2, filename=f"release_force_target_vs_ls3_rope_release", subdir="combined")

#### Zusammenhang Vorspannung und resultierende Lastspitzen

In [ ]:
fig3, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax2.axis('off')
y_pos_init = 0.1
for idx, (treatment, color) in enumerate(treatment_color_dict.items()):
    subset = df_id[df_id['treatment'] == treatment]
    if subset['ls3_cable_max'].isna().all():
        continue
    sns.regplot(x='ls3_rope_release', y='ls3_cable_max', data=subset, ax=ax1, color=color, label=treatment, ci=95)
    stats_text = annotate_stats(subset['ls3_rope_release'], subset['ls3_cable_max'])
    ax2.annotate(f"{treatment}:\n{stats_text}", xy=(1.01, y_pos_init + idx * 0.3), xycoords='axes fraction')
#ax1.set_title('Correlation Between Release Force and Peak Cable Force')
ax1.set_xlabel('Release Force [kN]')
ax1.set_ylabel('Peak Force in Cable [kN]')
ax1.legend(title='Treatment', loc='upper left')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig3, filename=f"release_force_vs_ls3_cable_max", subdir="combined")

In [ ]:
fig3, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax2.axis('off')
y_pos_init = 0.1
for idx, (treatment, color) in enumerate(treatment_color_dict.items()):
    subset = df_id[df_id['treatment'] == treatment]
    if subset['ptq_m_amplitude'].isna().all():
        continue
    sns.regplot(x='ls3_rope_release', y='ptq_m_amplitude', data=subset, ax=ax1, color=color, label=treatment,
                ci=95)
    stats_text = annotate_stats(subset['ls3_rope_release'], subset['ptq_m_amplitude'])
    ax2.annotate(f"{treatment}:\n{stats_text}", xy=(1.01, y_pos_init + idx * 0.3), xycoords='axes fraction')
#ax1.set_title('Correlation Between Release Force and Elongation Amplitude')
ax1.set_xlabel('Release Force [kN]')
ax1.set_ylabel('Elongation Amplitude [$\mu$m] (mean for all Sensors)')
ax1.legend(title='Treatment', loc='upper left')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig3, filename=f"ls3_release_force_vs_ptq_m_amplitude", subdir="combined")


In [ ]:
fig3, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax2.axis('off')
y_pos_init = 0.1
for idx, (treatment, color) in enumerate(treatment_color_dict.items()):
    subset = df_id[df_id['treatment'] == treatment]
    if subset['ptq_m_amplitude_2'].isna().all():
        continue
    sns.regplot(x='ls3_rope_release', y='ptq_m_amplitude_2', data=subset, ax=ax1, color=color, label=treatment,
                ci=95)
    stats_text = annotate_stats(subset['ls3_rope_release'], subset['ptq_m_amplitude_2'])
    ax2.annotate(f"{treatment}:\n{stats_text}", xy=(1.01, y_pos_init + idx * 0.3), xycoords='axes fraction')
#ax1.set_title('Correlation Between Release Force and Elongation Amplitude 2')
ax1.set_xlabel('Release Force [kN]')
ax1.set_ylabel('Elongation Amplitude 2 [$\mu$m] (mean for all Sensors)')
ax1.legend(title='Treatment', loc='upper left')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig3, filename=f"ls3_release_force_vs_ptq_m_amplitude_2", subdir="combined")

In [ ]:
# Funktion zur Durchführung des ANOVA-Tests und Berechnung der Effektstärke (Eta Squared)
def perform_anova_and_effect_size(df: pd.DataFrame, variable: str, treatments: List[str]) -> str:
    groups = [df[df['treatment'] == treatment][variable].dropna() for treatment in treatments]
    f_stat, p_value = f_oneway(*groups)

    # Berechnung der Effektstärke (Eta Squared)
    n = sum([len(g) for g in groups])
    ss_total = sum([(x - df[variable].mean()) ** 2 for g in groups for x in g])
    eta_squared = f_stat * len(groups) / (f_stat * len(groups) + (n - len(groups)))

    # Überprüfung der Signifikanz
    significance = "*" if p_value < 0.05 else ""

    return f"{variable}: {significance}\nF-statistic = {f_stat:.2f}\np-value = {p_value:.2e}\nEta Squared = {eta_squared:.2f}"

In [ ]:
# Funktion zur Erstellung von Boxplots
def create_boxplot(df: pd.DataFrame, variable: str, group_by: str, ax: plt.Axes, color_dict: Dict[str, str], perform_stats: bool) -> None:
    valid_df = df.dropna(subset=[variable])
    sns.boxplot(x=group_by, y=variable, hue=group_by, data=valid_df, ax=ax, palette=color_dict, dodge=False)
    ax2 = ax.twinx()
    ax2.axis('off')
    if perform_stats:
        stats_text = perform_anova_and_effect_size(valid_df, variable, valid_df[group_by].unique())
        ax2.annotate(stats_text, xy=(1.01, 0.1), xycoords='axes fraction')
    ax.set_title(f'Einfluss von {group_by} auf {variable}')
    ax.set_xlabel(group_by)
    ax.set_ylabel(variable)

# Funktion zur Erstellung kombinierter Plots
def create_combined_plot(df: pd.DataFrame, columns: List[str], group_by: str, color_dict: Dict[str, str], num_columns: int = 3, perform_stats: bool = False) -> None:
    num_rows = len(columns) // num_columns + (len(columns) % num_columns > 0)
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(16, 4 * num_rows))
    axes = axes.flatten()

    for idx, variable in enumerate(columns):
        create_boxplot(df, variable, group_by, axes[idx], color_dict, perform_stats)

    plt.tight_layout()
    plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"combined_plot_{group_by}", subdir="combined")

# Funktion zur Erstellung einzelner Plots
def create_individual_plots(df: pd.DataFrame, columns: List[str], group_by: str, color_dict: Dict[str, str], perform_stats: bool = False) -> None:
    for variable in columns:
        fig, ax = plt.subplots(figsize=(8, 5))
        create_boxplot(df, variable, group_by, ax, color_dict, perform_stats)
        plt.tight_layout()
        #plt.show()
        PLOT_MANAGER.save_plot(fig, filename=f"{group_by}_{variable}", subdir="individual_plots")

In [ ]:
columns = ['ptq_m_amplitude',
           'ptq_m_amplitude_2',
           'ptq_initial_amplitude',
           'ptq_damping_coeff',
           'ptq_angular_frequency',
           'ptq_y_shift',
           'ptq_pearson_r',
           #'ptq_nrmse',
           #'ptq_nmae',
           #'release_force_target',
           'ls3_rope_release',
           'ls3_cable_max'
           ]

# Beispiel: Erstellen von Plots gruppiert nach 'treatment'
create_combined_plot(df, columns, 'treatment', treatment_color_dict, perform_stats=True)
create_individual_plots(df, columns, 'treatment', treatment_color_dict, perform_stats=True)

In [ ]:
# Beispiel: Erstellen von Plots gruppiert nach 'ptq_sensor_name'
columns = ['ptq_m_amplitude', 'ptq_m_amplitude_2', 'ptq_initial_amplitude', 'ptq_damping_coeff', 'ptq_angular_frequency', 'ptq_pearson_r']

create_combined_plot(df, columns, 'ptq_sensor_name', sensor_color_dict)
create_individual_plots(df, columns, 'ptq_sensor_name', sensor_color_dict)

In [ ]:
fig3, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax2.axis('off')
y_pos_init = 0.1
for idx, (treatment, color) in enumerate(treatment_color_dict.items()):
    subset = df_id[df_id['treatment'] == treatment]
    if subset['ptq_m_amplitude_2'].isna().all():
        continue
    sns.regplot(x='ls3_rope_release', y='ptq_m_amplitude_2', data=subset, ax=ax1, color=color, label=treatment,
                ci=95)
    stats_text = annotate_stats(subset['ls3_rope_release'], subset['ptq_m_amplitude_2'])
    ax2.annotate(f"{treatment}:\n{stats_text}", xy=(1.01, y_pos_init + idx * 0.3), xycoords='axes fraction')
#ax1.set_title('Correlation Between Release Force and Elongation Amplitude 2')
ax1.set_xlabel('Release Force [kN]')
ax1.set_ylabel('Elongation Amplitude 2 [$\mu$m] (mean for all Sensors)')
ax1.legend(title='Treatment', loc='upper left')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig3, filename=f"ls3_release_force_vs_ptq_m_amplitude_2", subdir="combined")

In [ ]:
# Funktion zur Durchführung des ANOVA-Tests und Berechnung der Effektstärke (Eta Squared)
def perform_anova_and_effect_size(df: pd.DataFrame, variable: str, treatments: List[str]) -> str:
    groups = [df[df['treatment'] == treatment][variable].dropna() for treatment in treatments]
    f_stat, p_value = f_oneway(*groups)

    # Berechnung der Effektstärke (Eta Squared)
    n = sum([len(g) for g in groups])
    ss_total = sum([(x - df[variable].mean()) ** 2 for g in groups for x in g])
    eta_squared = f_stat * len(groups) / (f_stat * len(groups) + (n - len(groups)))

    # Überprüfung der Signifikanz
    significance = "*" if p_value < 0.05 else ""

    return f"{variable}: {significance}\nF-statistic = {f_stat:.2f}\np-value = {p_value:.2e}\nEta Squared = {eta_squared:.2f}"

In [ ]:
# Funktion zur Erstellung von Boxplots
def create_boxplot(df: pd.DataFrame, variable: str, group_by: str, ax: plt.Axes, color_dict: Dict[str, str], perform_stats: bool) -> None:
    valid_df = df.dropna(subset=[variable])
    sns.boxplot(x=group_by, y=variable, hue=group_by, data=valid_df, ax=ax, palette=color_dict, dodge=False)
    ax2 = ax.twinx()
    ax2.axis('off')
    if perform_stats:
        stats_text = perform_anova_and_effect_size(valid_df, variable, valid_df[group_by].unique())
        ax2.annotate(stats_text, xy=(1.01, 0.1), xycoords='axes fraction')
    ax.set_title(f'Einfluss von {group_by} auf {variable}')
    ax.set_xlabel(group_by)
    ax.set_ylabel(variable)

# Funktion zur Erstellung kombinierter Plots
def create_combined_plot(df: pd.DataFrame, columns: List[str], group_by: str, color_dict: Dict[str, str], num_columns: int = 3, perform_stats: bool = False) -> None:
    num_rows = len(columns) // num_columns + (len(columns) % num_columns > 0)
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(16, 4 * num_rows))
    axes = axes.flatten()

    for idx, variable in enumerate(columns):
        create_boxplot(df, variable, group_by, axes[idx], color_dict, perform_stats)

    plt.tight_layout()
    plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"combined_plot_{group_by}", subdir="combined")

# Funktion zur Erstellung einzelner Plots
def create_individual_plots(df: pd.DataFrame, columns: List[str], group_by: str, color_dict: Dict[str, str], perform_stats: bool = False) -> None:
    for variable in columns:
        fig, ax = plt.subplots(figsize=(8, 5))
        create_boxplot(df, variable, group_by, ax, color_dict, perform_stats)
        plt.tight_layout()
        #plt.show()
        PLOT_MANAGER.save_plot(fig, filename=f"{group_by}_{variable}", subdir="individual_plots")

In [ ]:
columns = ['ptq_m_amplitude',
           'ptq_m_amplitude_2',
           'ptq_initial_amplitude',
           'ptq_damping_coeff',
           'ptq_angular_frequency',
           'ptq_y_shift',
           'ptq_pearson_r',
           #'ptq_nrmse',
           #'ptq_nmae',
           #'release_force_target',
           'ls3_rope_release',
           'ls3_cable_max'
           ]

# Beispiel: Erstellen von Plots gruppiert nach 'treatment'
create_combined_plot(df, columns, 'treatment', treatment_color_dict, perform_stats=True)
create_individual_plots(df, columns, 'treatment', treatment_color_dict, perform_stats=True)

In [ ]:
# Beispiel: Erstellen von Plots gruppiert nach 'ptq_sensor_name'
columns = ['ptq_m_amplitude', 'ptq_m_amplitude_2', 'ptq_initial_amplitude', 'ptq_damping_coeff', 'ptq_angular_frequency', 'ptq_pearson_r']

create_combined_plot(df, columns, 'ptq_sensor_name', sensor_color_dict)
create_individual_plots(df, columns, 'ptq_sensor_name', sensor_color_dict)